In [1]:
"""
This notebook walks though how to use BuildingMOTIF Templates.
"""
from rdflib import RDF, URIRef

from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library, Template
from buildingmotif.namespaces import BRICK, A, PARAM

In [2]:
"""
As always with BuildingMOTIF, we start by initing BuildingMOTIF.
"""
building_motif = BuildingMOTIF(f"sqlite://")
building_motif.session


In [3]:
"""
Templates must always be conneted to a library.
"""
lib = Library.create("my_lib")
lib

Library(_id=1, _name='my_lib', _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x10515db50>)

In [4]:
"""
Create a template through the library. The body is currently empty.

"""
ahu_template = lib.create_template(
    name="ahu",
)

print(ahu_template)
print(ahu_template.body.serialize(format="ttl"))

Template(_id=1, _name='ahu', body=<Graph identifier=bee406a0-a64a-4e7a-bb90-7ac3229bd347 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x10515db50>)




In [5]:
"""
Now we can populate the body.
Note that the nodes "name" and "zone" are variables, or undefinied.
They are the template's parameters, as defined above. This means their values
will be pass to the template when evaluated.
"""

ahu_template.body.add((PARAM["name"], RDF.type, BRICK["AHU"]))
ahu_template.body.add((PARAM["name"], BRICK["feeds"], PARAM["zone"]))


print(ahu_template.body.serialize(format="ttl"))

@prefix P: <urn:___param___#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .

P:name a brick:AHU ;
    brick:feeds P:zone .




In [6]:
"""
Note that the parameters are immutable.
"""
try:
    ahu_template.parameters = ["nope"]
except AttributeError as e:
    print(e)

try:
    ahu_template.parameters.append("nope")
except AttributeError as e:
    print(e)

property 'parameters' of 'Template' object has no setter
'set' object has no attribute 'append'


In [7]:
"""
In addition to parameters, templates may also have dependancies on other templates.
Say we added to ahu_template the following tuple, where "sf" refers to the
following supply_fan template.

We must then create a dependancy between the two, and define how the dependancy
parameters should be evaluated.
"""
# this new triples requires dependancy "sf"
ahu_template.body.add((PARAM["name"], BRICK["hasPart"], PARAM["sf"]))

# here we define the template we will depend on
sf_template = lib.create_template(
    name="supply_fan",
)
sf_template.body.add((PARAM["name"], RDF.type, BRICK["Supply_Fan"]))

# and finally define the dependancy.
ahu_template.add_dependency(sf_template, {"name": "sf"})

print(ahu_template.get_dependencies())

(Dependency(_template_id=2, args={'name': 'sf'}),)


In [8]:
""" 
Note that both templates must be part of the same library, 
all parameters must be defined to create a dependancy,
and that there can only be one dependency per directional dependant/dependee pair.
"""
building_motif.session.commit()

# not in same library
try:
    other_lib = lib = Library.create("other_lib")
    other_sf_template = lib.create_template(
        name="supply_fan",
    )
    ahu_template.add_dependency(other_sf_template, {"name": "sf"})
except Exception as e:
    print(e)
    building_motif.session.rollback()
    
# bad required_params
try:
    ahu_template.add_dependency(sf_template, {"not a required_params": "sf"})
except ValueError as e:
    print(e)

# already exist
try:
    ahu_template.add_dependency(sf_template, {"name": "sf"})
except Exception as e:
    print(e)
    building_motif.session.rollback()


The name parameter is required for the dependency 'ahu'.
(sqlite3.IntegrityError) UNIQUE constraint failed: deps_association_table.dependant_id, deps_association_table.dependee_id, deps_association_table.args
[SQL: INSERT INTO deps_association_table (dependant_id, dependee_id, args) VALUES (?, ?, ?)]
[parameters: (1, 2, '[["name", "sf"]]')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


# Loading Templates from an Ontology

There is initial support for loading templates from the Class and NodeShape definitions contained within an ontology:

In [9]:
lib = Library.load(directory="../tests/unit/fixtures/Brick1.3rc1-equip-only.ttl")
print(lib)

Library(_id=2, _name='Brick1.3rc1-equip-only.ttl', _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x10515db50>)


/Users/gabe/Library/Caches/pypoetry/virtualenvs/buildingmotif-ozGZrh9J-py3.11/lib/python3.11/site-packages/pyshacl/extras/__init__.py:46: Warning: Extra "js" is not satisfied because requirement pyduktape2 is not installed.
  warn(Warning(f"Extra \"{extra_name}\" is not satisfied because requirement {req} is not installed."))


In [10]:
"""TO COME: template evaluation"""

'TO COME: template evaluation'